### Imports

In [1]:
import sys
sys.dont_write_bytecode = True


import torch
import numpy as np
import random
import os

def set_seeds(seed_value=42):
    """Sets seeds for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)

set_seeds(42) 

import json
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import warnings
import logging
from datetime import datetime
warnings.filterwarnings('ignore')

from model import get_model
from config import CFG
from dataset import get_dataset_class
# from plot import plot_domainwise_accuracy
from transform import get_transforms
from runner import run_baseline, run_lodo

torch.manual_seed(CFG["system"]["seed"])
np.random.seed(CFG["system"]["seed"])

device = CFG["system"]["device"]
print(f"Device: {device}")
print(f"PyTorch: {torch.__version__}")

DS = "CMNIST"
MODEL_NAME = "WinKawaks/vit-small-patch16-224"

c:\Users\Fatim_Sproj\anaconda3\envs\bacp\lib\site-packages\torch\cuda\__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


Device: cuda
PyTorch: 2.5.1+cu121


### DataLoading

In [2]:
train_transform, test_transform = get_transforms(img_size=224, augment=False, use_imagenet_norm=False)

DatasetClass = get_dataset_class(DS)

ld = DatasetClass(
    data_root=CFG["datasets"][DS]["root"],
    transform=train_transform,
    batch_size=CFG["train"]["batch_size"]
)

print("\nData loaders ready!")


Data loaders ready!


### Logging

In [3]:
dataset_name = DS
base_dir = os.path.join(os.getcwd(), dataset_name)
subdirs = ["logs", "checkpoints", "plots"]

for sub in subdirs:
    os.makedirs(os.path.join(base_dir, sub), exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_file = os.path.join(base_dir, "logs", f"train_{timestamp}.log")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(f"{dataset_name}_logger")

logger.info(f"Initialized experiment directories for {dataset_name}")
logger.info(f"Logs: {os.path.join(base_dir, 'logs')}")
logger.info(f"Checkpoints: {os.path.join(base_dir, 'checkpoints')}")
logger.info(f"Plots: {os.path.join(base_dir, 'plots')}")

2025-10-21 07:04:16,066 | INFO | Initialized experiment directories for CMNIST
2025-10-21 07:04:16,066 | INFO | Logs: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-small\CMNIST\logs
2025-10-21 07:04:16,066 | INFO | Checkpoints: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-small\CMNIST\checkpoints
2025-10-21 07:04:16,066 | INFO | Plots: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-small\CMNIST\plots


### Setup

In [4]:
domains = CFG["datasets"][DS]["domains"]
loaders = {d: {"train": ld.get_dataloader(d, train=True), "val": ld.get_dataloader(d, train=False)} for d in domains}
ckpt_root = os.path.join(base_dir, "checkpoints")
log_dir = os.path.join(base_dir, "logs")
plots_dir = os.path.join(base_dir, "plots")
os.makedirs(ckpt_root, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)
model_factory = lambda cfg, dataset_key: get_model(cfg,dataset=DS)
optimizer_fn = lambda model: optim.AdamW(model.parameters(), lr=CFG["train"]["lr"], weight_decay=CFG["train"].get("weight_decay", 0.01))
device = CFG["system"]["device"]
epochs = CFG["train"]["epochs"]


{
  "lodo_results": {
    "art_painting": 0.8341463414634146,
    "cartoon": 0.7974413646055437,
    "photo": 0.9580838323353293,
    "sketch": 0.6017811704834606
  },
  "timestamp": "20251004_020611"
}

### Leave One Domain Out

In [5]:
lodo_results, lodo_mean, lodo_summary = run_lodo(
    model_fn=model_factory,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    ckpt_root=ckpt_root,
    log_dir=log_dir,
    epochs=epochs
)

Some weights of ViTModel were not initialized from the model checkpoint at WinKawaks/vit-small-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-10-21 04:26:19,559 | INFO | === LODO: Leaving out domain 'red' ===



=== LODO: Leaving out domain 'red' ===


Evaluating: 100%|██████████| 50/50 [00:12<00:00,  3.93it/s]
2025-10-21 04:27:05,566 | INFO | [red] Epoch 1/5 | Train - Loss: 0.5243, Cls: 0.5187, GRQO: 0.0056, Acc: 0.8336 | Val - Loss: 0.1043, Cls: 0.1039, GRQO: 0.0004, Acc: 0.9704
2025-10-21 04:27:05,756 | INFO | [red] New best val acc: 0.9704


[red] Epoch 1/5 | Train - Loss: 0.5243, Cls: 0.5187, GRQO: 0.0056, Acc: 0.8336 | Val - Loss: 0.1043, Cls: 0.1039, GRQO: 0.0004, Acc: 0.9704
[red] New best val acc: 0.9704


Evaluating: 100%|██████████| 50/50 [00:12<00:00,  4.17it/s]
2025-10-21 04:28:08,633 | INFO | [red] Epoch 2/5 | Train - Loss: 0.0648, Cls: 0.0624, GRQO: 0.0024, Acc: 0.9812 | Val - Loss: 0.0766, Cls: 0.0764, GRQO: 0.0002, Acc: 0.9762
2025-10-21 04:28:08,828 | INFO | [red] New best val acc: 0.9762


...

[blue] Epoch 4/5 | Train - Loss: 0.0270, Cls: 0.0254, GRQO: 0.0016, Acc: 0.9920 | Val - Loss: 0.0513, Cls: 0.0512, GRQO: 0.0001, Acc: 0.9862


2025-10-21 04:40:34,090 | INFO | [blue] New best val acc: 0.9862


[blue] New best val acc: 0.9862


Evaluating: 100%|██████████| 49/49 [00:12<00:00,  3.94it/s]
2025-10-21 04:41:36,806 | INFO | [blue] Epoch 5/5 | Train - Loss: 0.0169, Cls: 0.0155, GRQO: 0.0014, Acc: 0.9955 | Val - Loss: 0.0594, Cls: 0.0593, GRQO: 0.0000, Acc: 0.9836
2025-10-21 04:41:36,806 | INFO | [blue] Best Acc: 0.9862
2025-10-21 04:41:36,806 | INFO | ------------------------------------------------------------
2025-10-21 04:41:36,806 | INFO | LODO finished | Mean Acc: 0.9855 | Summary saved to c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-small\CMNIST\logs\lodo_summary_20251021_044136.json


[blue] Epoch 5/5 | Train - Loss: 0.0169, Cls: 0.0155, GRQO: 0.0014, Acc: 0.9955 | Val - Loss: 0.0594, Cls: 0.0593, GRQO: 0.0000, Acc: 0.9836
[blue] Best Acc: 0.9862
------------------------------------------------------------
LODO finished | Mean Acc: 0.9855
Summary saved to c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-small\CMNIST\logs\lodo_summary_20251021_044136.json


### Baseline

In [5]:
baseline_results, baseline_mean = run_baseline(
    model_name=MODEL_NAME,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    epochs=CFG["train"]["epochs"]
)

2025-10-21 07:04:18,132 | INFO | Initializing ViT baseline: WinKawaks/vit-small-patch16-224


Initializing ViT baseline: WinKawaks/vit-small-patch16-224


Some weights of ViTForImageClassification were not initialized from the model checkpoint at WinKawaks/vit-small-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 384]) in the checkpoint and torch.Size([10, 384]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-10-21 07:04:19,098 | INFO | === Baseline LODO: Leaving out domain 'red' ===



=== Baseline LODO: Leaving out domain 'red' ===


2025-10-21 07:04:58,814 | INFO | [red] Epoch 1/5 | Train - Loss: 0.3018, Acc: 0.9065 | Val Acc: 0.9687


...

[blue] Epoch 3/5 | Train - Loss: 0.0249, Acc: 0.9924 | Val Acc: 0.9877


2025-10-21 07:13:43,229 | INFO | [blue] Epoch 4/5 | Train - Loss: 0.0198, Acc: 0.9936 | Val Acc: 0.9846


[blue] Epoch 4/5 | Train - Loss: 0.0198, Acc: 0.9936 | Val Acc: 0.9846


2025-10-21 07:14:24,014 | INFO | [blue] Epoch 5/5 | Train - Loss: 0.0145, Acc: 0.9948 | Val Acc: 0.9856
2025-10-21 07:14:24,014 | INFO | [blue] Best Val Acc: 0.9877
2025-10-21 07:14:24,014 | INFO | ------------------------------------------------------------
2025-10-21 07:14:24,014 | INFO | Baseline LODO (WinKawaks/vit-small-patch16-224) finished | Mean Acc: 0.9860


[blue] Epoch 5/5 | Train - Loss: 0.0145, Acc: 0.9948 | Val Acc: 0.9856
[blue] Best Val Acc: 0.9877
------------------------------------------------------------
Baseline LODO (WinKawaks/vit-small-patch16-224) finished | Mean Acc: 0.9860
